In [2]:
%set_env PYTORCH_CUDA_ALLOC_CONF=backend:cudaMallocAsync

env: PYTORCH_CUDA_ALLOC_CONF=backend:cudaMallocAsync


In [3]:
from langchain.document_loaders import PDFPlumberLoader, DirectoryLoader
from langchain.document_loaders.word_document import Docx2txtLoader
from langchain.document_loaders.unstructured import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
import os

cached_db = True
loader_type = "docx"
loader = None

print(Chroma)
if not cached_db:
    if loader_type == "pdf":
        loader_cls, glob = PDFPlumberLoader, "*.pdf"
    elif loader_type == "docx":
        loader_cls, glob = Docx2txtLoader, "*.docx"
    elif loader_type == "txt":
        loader_cls, glob = UnstructuredFileLoader, "*.txt"
    else:
        loader_cls, glob = None, None
    loader = DirectoryLoader(
        "data/MGRP", glob=glob, loader_cls=loader_cls, show_progress=True
    )
    pages = loader.load_and_split()

<class 'langchain.vectorstores.chroma.Chroma'>


In [4]:
import importer
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings
import tiktoken
from langchain_extras.llms.exllama import ExLlama, BasicStreamingHandler
from langchain.chains import loading, RetrievalQA, VectorDBQA
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser


def print_attributes(obj):
    attributes = dir(obj)
    for attr in attributes:
        print(attr)


handler = BasicStreamingHandler()

encoding = tiktoken.get_encoding("cl100k_base")
total_token_count = 0
model_directory = "models/TheBloke_GPT4All-13B-Snoozy-SuperHOT-8K-GPTQ"

llm = ExLlama(
    streaming=True,
    model_path=model_directory,
    lora_path=None,
    temperature=0.7,
    beams=1,
    beam_length=40,
    stop_sequences=["Human:", "User:", "AI:"],
    callbacks=[handler],
    verbose=True,
    max_seq_len=4096,
    alpha_value=4.0,  # For use with any models
    compress_pos_emb=4.0,  # For use with superhot
    # set_auto_map = "3, 2" #Gpu split, this will split 3gigs/2gigs
)


def tiktoken_length(text):
    global total_token_count
    token_count = encoding.encode(text)
    if len(token_count) == 0:
        return 0
    else:
        total_token_count += token_count[0]
        return token_count[0]


embeddings = HuggingFaceEmbeddings(model_name="all-mpnet-base-v2")

if cached_db:
    db = Chroma(persist_directory="data", embedding_function=embeddings)
    print_attributes(db)
else:
    db = Chroma.from_documents(pages, embeddings, persist_directory="data")
    db.persist()


class QuestionWithAnswer(BaseModel):
    answer: str = Field(description="The answer to the question")


parser = PydanticOutputParser(pydantic_object=QuestionWithAnswer)

story_description = """
The hit smartphone game Magical Girl Raising Project is not only free, but one in tens of thousands of players 
will win real-life powers. In N-City, sixteen of these lucky girls help the city's citizens - until upper 
management announces that each week, the least productive Magical Girl will lose her gifts. But this is no 
ordinary contest, and as the rules become increasingly sadistic, 
the competition to keep their powers becomes a vicious battle for the girls' very lives.

Each girl in the competition has a real name and a magical girl name. An example of a real name is Murota Tsubame 
and her magical girl name is Top Speed.

There are some characters who are not magical girls. 

"""
prompt_template = """
The following context is the text from the 1st series of a Japanese light novel called Magical Girl Raising Project. 

{format_instructions}

Here is a description of the story you will be expected to answer questions about:

{description}

Context:

{context}

Question: {question}
Answer:"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"],
    partial_variables={
        "description": story_description,
        "format_instructions": parser.get_format_instructions(),
    },
)
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(),
    chain_type_kwargs={"prompt": prompt},
    verbose=False,
)

handler.set_chain(chain.combine_documents_chain)
print("\n")

/home/jovyan/work/importer.py
max_seq_len 4096
compress_pos_emb 4.0
alpha_value 4.0
temperature 0.7
top_k 40
top_p 0.65
token_repetition_penalty_max 1.15
token_repetition_penalty_sustain 256
token_repetition_penalty_decay 128
beams 1
beam_length 40
stop_sequences ['human:', 'user:', 'ai:']


2023-07-06 19:52:55.503824: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-06 19:52:55.705687: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-07-06 19:52:56.364781: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-07-06 19:52:56.364909: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

_Chroma__query_collection
_LANGCHAIN_DEFAULT_COLLECTION_NAME
__abstractmethods__
__annotations__
__class__
__delattr__
__dict__
__dir__
__doc__
__eq__
__format__
__ge__
__getattribute__
__gt__
__hash__
__init__
__init_subclass__
__le__
__lt__
__module__
__ne__
__new__
__reduce__
__reduce_ex__
__repr__
__setattr__
__sizeof__
__slots__
__str__
__subclasshook__
__weakref__
_abc_impl
_client
_client_settings
_collection
_embedding_function
_persist_directory
_similarity_search_with_relevance_scores
aadd_documents
aadd_texts
add_documents
add_texts
afrom_documents
afrom_texts
amax_marginal_relevance_search
amax_marginal_relevance_search_by_vector
as_retriever
asearch
asimilarity_search
asimilarity_search_by_vector
asimilarity_search_with_relevance_scores
delete
delete_collection
from_documents
from_texts
get
max_marginal_relevance_search
max_marginal_relevance_search_by_vector
persist
search
similarity_search
similarity_search_by_vector
similarity_search_with_relevance_scores
similarity_sea

In [5]:
# parser.parse(
chain.run("What is this story about?")
chain.run(
    """
    Using the provided document embeddings of the story, generate a list of characters along with their respective names and any known aliases.

    """
)
# )
# chain.run("What is the name of the main character?")
# chain.run("What is the real world name of Snow White?")
# chain.run("Which character is pregnant?")
# chain.run("Who is the primary antagonist?")
# chain.run("Which magical girl is the first to die?")


The following context is the text from the 1st series of a Japanese light novel called Magical Girl Raising Project. 

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"answer": {"title": "Answer", "description": "The answer to the question", "type": "string"}}, "required": ["answer"]}
```

Here is a description of the story you will be expected to answer questions about:


The hit smartphone game Magical Girl Raising Project is not only free, but one in tens of thousands of players 
will win real-life powers. In N-City, sixteen of these lucky girls help the city's citizens

'\n```json\n{\n"characters": [\n{\n"name": "Murota Tsubame",\n"alias": ""\n},\n{\n"name": "Snow White",\n"alias": ""\n}\n]\n}\n```'